# Semi-invertible autoencoder

In [1]:
import numpy as np
import logging
import sys
from matplotlib import pyplot as plt
from sklearn.manifold import TSNE

%matplotlib inline
sys.path.append("../")

import torch
from torchvision.datasets import MNIST
from torchvision import transforms

from aef.models.autoencoding_flow import TwoStepAutoencodingFlow
from aef.trainer import AutoencodingFlowTrainer
from aef.losses import nll, mse

logging.basicConfig(
    format="%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s",
    datefmt="%H:%M",
    level=logging.INFO,
)


## Data

In [2]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
mnist = MNIST('./data', download=True, transform=img_transform)

## Train autoencoder

In [3]:
ae = TwoStepAutoencodingFlow(data_dim=28*28, latent_dim=10, steps_inner=5, steps_outer=5)

12:14 aef.models.autoencod INFO    Created autoencoding flow with 18614690 trainable parameters


In [ ]:
trainer = AutoencodingFlowTrainer(ae, output_filename="../figures/training/aef/aef_phase1")
trainer.train(
    dataset=mnist,
    loss_functions=[mse],
    loss_labels=["MSE"],
    loss_weights=[1.],
    batch_size=256,
    epochs=5,
    verbose="all",
    initial_lr=1.e-3,
    final_lr=1.e-4,
)

In [ ]:
trainer = AutoencodingFlowTrainer(ae, output_filename="../figures/training/aef/aef_phase2")
trainer.train(
    dataset=mnist,
    loss_functions=[mse, nll],
    loss_labels=["MSE", "NLL"],
    loss_weights=[1., 0.01],
    batch_size=256,
    epochs=5,
    verbose="all",
    initial_lr=1.e-3,
    final_lr=1.e-4,
    parameters=ae.outer_transform.parameters()
)

## Visualize latent space

In [ ]:
n = 1000

x = torch.cat([mnist[i][0].unsqueeze(0) for i in range(n)], dim=0)
y = np.asarray([mnist[i][1] for i in range(n)])

x = x.view(x.size(0), -1)
x.to(torch.device("cpu"))
x_out, _, u = ae(x)

x = x.detach().numpy().reshape(-1, 28, 28)
x_out = x_out.detach().numpy().reshape(-1, 28, 28)
u = u.detach().numpy().reshape(x_out.shape[0], -1)
tsne = TSNE(n_components=2, verbose=0, perplexity=40, n_iter=300).fit_transform(u)

In [ ]:
fig = plt.figure(figsize=(10,5))

ax = plt.subplot(1,2,1)
for i in range(10):
    plt.scatter(u[y==i][:,0], u[y==i][:,1], c="C{}".format(i), s=20., label="{}".format(i+1))
plt.legend()
plt.xlabel(r"$u_0$")
plt.ylabel(r"$u_1$")
    
ax = plt.subplot(1,2,2)
for i in range(10):
    plt.scatter(tsne[y==i][:,0], tsne[y==i][:,1], c="C{}".format(i), s=20., label="{}".format(i+1))
plt.legend()
plt.xlabel(r"$t-SNE component 0$")
plt.ylabel(r"$t-SNE component 1$")

plt.tight_layout()
plt.savefig("../figures/mnist_latent.pdf")


## Visualize reconstruction

In [ ]:
x = torch.cat([mnist[i][0] for i in range(1000)], dim=0)
x = x.view(x.size(0), -1)
y = np.asarray([mnist[i][1] for i in range(1000)])

In [ ]:
x.size()

In [ ]:
h = ae.encode(x)
x_out = ae.decode(h)

In [ ]:
h = h.detach().numpy()
x = x.detach().numpy()
x_out = x_out.detach().numpy()

In [ ]:
fig = plt.figure(figsize=(12,12))

for i in range(18):
    ax = plt.subplot(6, 6, 2*i + 1)
    plt.imshow(x[i].reshape((28,28)), vmin=-1.1, vmax=1.1)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    ax = plt.subplot(6, 6, 2*i + 2)
    plt.imshow(x_out[i].reshape((28,28)), vmin=-1.1, vmax=1.1)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
        
plt.tight_layout()
plt.savefig("../figures/reconstruction.pdf")
